In [1]:
import random
import time
import numpy as np
from sys import stderr

In [2]:
def create_sys2():
    du = np.array([0.0475, 0.0523, 0.057])
    d = np.array([10.8, 9.9, 9.0, 8.1])
    dl = np.array([0.0, 0.0321, 0.0369, 0.0416])
    b = np.array([12.1430, 13.0897, 13.6744, 13.8972])
    
    return du, d, dl, b

In [3]:
# def create_sys2():
#     N = 100
#     d = 2+np.random.rand(N) # d от 2 до 3
#     du = -np.random.rand(N) # du от -1 до 0
#     dl = -np.random.rand(N) # dl от -1 до 0
#     b = 2*np.random.rand(N)-1 # b от -1 до 1

#     return du, d, dl, b
    

In [4]:
def thomas_solver2( du, d, dl, h):
    if d.all() == 0 or d[0]==0:
        stderr.write('rewrite matrix A')
        exit(-1)
    
    N = len(d)

    q = np.zeros(N)
    g = np.zeros(N)
    w = np.zeros(N)  
    u = np.zeros(N)
    
    # forward
    q[0] = du[0] / d[0]
    g[0] = h[0] / d[0]


    
    for i in range(1, N):
        w[i] = d[i] - dl[i] * q[i - 1]
        if w[i] == 0:
            q[i] = 0
            g[i] = 0
            
        if i != N - 1:
            q[i] = du[i] / w[i]
        
        g[i] = (h[i] - dl[i] * g[i - 1]) / w[i]

    # backward
    u[N - 1] = g[N - 1]
    
    for i in range(N - 2, -1, -1):  
        u[i] = g[i] - q[i] * u[i + 1]

    return u

In [5]:
du, d, dl, b = create_sys2()
print(type(b))

a = len(d)          
A = [0] * a 
for i in range(a): 
    A[i] = [0] * a 
print(A)

for i in range(a):
    A[i][i] = d[i]
    
for i in range(a-1):  
     for j in range(a):
        j = i
        A[i][j+1] = du[i] 

for i in range(1,a):  
     for j in range(a):
        j = i
        A[i][j-1] = dl[i] 

print(A)


<class 'numpy.ndarray'>
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[[10.8, 0.0475, 0, 0], [0.0321, 9.9, 0.0523, 0], [0, 0.0369, 9.0, 0.057], [0, 0, 0.0416, 8.1]]


In [6]:
start_time = time.time()
linalg_solution = np.linalg.solve(A, b)
print(linalg_solution)
end_time = time.time()
print('time of np.linalg.solve: ',(time.time() - start_time)/100, 'ms')

start_time = time.time()
my_solution = thomas_solver2(du, d, dl, b)
print("my_solution = ", my_solution)
end_time = time.time()
print('time of my solver: ', (time.time() - start_time) / 100, 'ms')

norm = 0
for i in range(len(my_solution)):
    if np.abs(my_solution[i] - linalg_solution[i]) > norm:
        norm = abs(my_solution[i] - linalg_solution[i])
print('norm: ', norm)


[1.11858753 1.31062391 1.50318699 1.70798363]
time of np.linalg.solve:  0.00015101194381713867 ms
my_solution =  [1.11858753 1.31062391 1.50318699 1.70798363]
time of my solver:  0.0 ms
norm:  2.220446049250313e-16


In [7]:
r = np.matmul(A,my_solution) - b  # невязка
r

array([-1.77635684e-15,  0.00000000e+00, -3.55271368e-15,  0.00000000e+00])

In [8]:
print("L2-norm: ", np.linalg.norm(r))

L2-norm:  3.972054645195637e-15
